In [10]:

import cv2
import os
from monlam_ocr.exceptions import FailedToLoadImage
from monlam_ocr.ocr import OCRInference


In [11]:

def run_ocr(
    image_path: str, ocr_inference: OCRInference
) -> None:
    image = [cv2.imread(image_path)]
    if image is None:
        FailedToLoadImage(f"Failed to load image from {image_path}")
    
    
    predicted_text, prediction = ocr_inference.run(image)
    return predicted_text

In [12]:

ocr_model_config_path = "/home/gangagyatso/Desktop/project16/image-to-text/Models/OCRModels/DergeTenjur/model_config.json"
ocr_inference = OCRInference(ocr_model_config_path)


KeyError: 'cpu-model'

In [ ]:
image_path = "/home/gangagyatso/Desktop/project16/image-to-text/data/input/Images_Part_5/I3CN520023_0.jpg"
predected_text = run_ocr(image_path, ocr_inference)
print(predected_text[0])

(1, 1, 80, 2000)
ཏལཔཏ༈༈ཨོབོཧ༈ཏལཔཅ༈ཧཏྭོཧ༈ཏལསཔས༈ཧཏབོཧ༈ཅབ༈ཅམབ༈ཨོབ༈ནཕེ༹༈རབ༼ཇ༈ཏལབས༈ཨེབོཧ༈ཅ༑ཨིཇས༈ཅཟཕ༹ཏ༈ཨོབཏ༈ཏལཔས༈ཨེབོཧ༈རཔ༈ཧཏཝན༈ཨོཇ༠༈ཅལསབ༈ཧཏ༈ཀཇ༈ཏལབ༼༈མུབ༈ལིབོཧ༈ཧཇ༈ལབ༈ཅབ༼ཇ༈ཅབཏབན༈མབས༈ཏཅསབཏ༈བནབ༈ཨོ༈ཅདབོཧཏ༈རབ༈མབཏ༈ནླིབ༼༈༼ཧསཔནཏ༈མབོཧ༈ཅཏུབོ༈ཨོབཏ༈ཨེཕ༹༈བོཧ༈


In [16]:
from botok import WordTokenizer
wt = WordTokenizer()
text = "ཀྲུའོ་ཕུའུ་ཕྲུའུ། (29)འདི་ཆགས་ནས་ལོ་༡༠༣ཙམ་སོང་ཡོད་ཚོད།"
tokens = wt.tokenize(text)
for token in tokens:
    print(token)


Loading Trie... (1s.)
text: "ཀྲུའོ་"
text_cleaned: "ཀྲུའོ་"
text_unaffixed: "ཀྲུའོ་"
syls: ["ཀྲུའོ"]
pos: NON_WORD
senses: | pos: NON_WORD |
char_types: |CONS|SUB_CONS|VOW|CONS|VOW|TSEK|
chunk_type: TEXT
syls_idx: [[0, 1, 2, 3, 4]]
syls_start_end: [{'start': 0, 'end': 6}]
start: 0
len: 6


text: "ཕུའུ་"
text_cleaned: "ཕུའུ་"
text_unaffixed: "ཕུའུ་"
syls: ["ཕུའུ"]
pos: NON_WORD
senses: | pos: NON_WORD |
char_types: |CONS|VOW|CONS|VOW|TSEK|
chunk_type: TEXT
syls_idx: [[0, 1, 2, 3]]
syls_start_end: [{'start': 0, 'end': 5}]
start: 6
len: 5


text: "ཕྲུའུ"
text_cleaned: "ཕྲུའུ་"
text_unaffixed: "ཕྲུའུ་"
syls: ["ཕྲུའུ"]
pos: NON_WORD
senses: | pos: NON_WORD |
char_types: |CONS|SUB_CONS|VOW|CONS|VOW|
chunk_type: TEXT
syls_idx: [[0, 1, 2, 3, 4]]
syls_start_end: [{'start': 0, 'end': 5}]
start: 11
len: 5


text: "། "
char_types: |NORMAL_PUNCT|TRANSPARENT|
chunk_type: PUNCT
start: 16
len: 2


text: "(29)"
char_types: |LATIN|LATIN|LATIN|LATIN|
chunk_type: LATIN
start: 18
len: 4


text: "འདི་"
text

In [19]:
pip install Levenshtein


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [40]:
import json
import os
import glob
import Levenshtein


def calculate_similarity(ocr_text, rearranged_text):
    # Ensure that neither string is None
    ocr_text = ocr_text if ocr_text is not None else ""
    rearranged_text = rearranged_text if rearranged_text is not None else ""

    # Calculate Levenshtein distance
    similarity = Levenshtein.ratio(ocr_text, rearranged_text)

    return similarity


def process_folder(folder_path):
    accuracies = []  # List to hold the accuracies of all items

    for work_folder in os.listdir(folder_path):
        work_folder_path = os.path.join(folder_path, work_folder)
        if not os.path.isdir(work_folder_path):
            continue  # Skip files, process only directories

        json_file_path = os.path.join(work_folder_path, f"rearranged_{work_folder}.json")
        if not os.path.exists(json_file_path):
            print(f"JSON file does not exist: {json_file_path}")
            continue

        with open(json_file_path, 'r', encoding='utf-8') as json_file:
            data = json.load(json_file)

        for item in data:
            try:
                ocr_text = item.get("text", "")
                rearranged_text = item.get("rearranged_text", "")
                if ocr_text is None or rearranged_text is None:
                    raise ValueError("One of the texts is None")

                similarity = calculate_similarity(ocr_text, rearranged_text)
                accuracies.append(similarity)

            except ValueError as e:
                print(f"Error processing item: {e}")
                print(f"Image Name: {item.get('image_name')}, Book ID: {item.get('book_id')}")

    # Calculate the average accuracy
    average_accuracy = sum(accuracies) / len(accuracies) if accuracies else 0
    return average_accuracy

# Example usage
folder_path = '/home/gangagyatso/Desktop/project16/image-to-text/data/input_rearranged_pytesseract'
average_accuracy = process_folder(folder_path)
print(f"Average OCR accuracy: {average_accuracy:.2f}%")


Average OCR accuracy: 0.88%
